In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Exploring Data

In [2]:
# Load data
df = pd.read_excel('ride_requests_2020.xlsx')

df.head(10)

,Request Creation Date,Request Creation Time,Request ID,Request Status,Rider ID,Booking Method,Destination Lat,Destination Lng,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Ride ID,Cancellation Time,Actual Pickup Time,Actual Dropoff Time,Ride Distance,Ride Duration,Ride Rating (1-5)
0,2020-01-01,2020-01-01 04:43:01,994012,Completed,43,Application,42.326003,-71.082605,2020-01-01 04:47:10,2020-01-01 05:04:05,4.14,72.16,41594.0,NaN,2020-01-01 04:45:41,2020-01-01 04:53:11,1.78,7.50,5.0
1,2020-01-01,2020-01-01 21:53:47,994065,Completed,1728,Application,42.322582,-71.109085,2020-01-01 21:59:48,2020-01-01 22:14:41,6.00,49.20,41610.0,NaN,2020-01-01 22:02:17,2020-01-01 22:10:08,1.82,7.85,5.0
2,2020-01-02,2020-01-02 00:55:31,994109,Completed,1118,Application,42.329393,-71.104067,2020-01-02 01:01:32,2020-01-02 01:10:56,6.00,49.20,41634.0,NaN,2020-01-02 00:59:24,2020-01-02 01:06:06,1.71,6.70,5.0
3,2020-01-03,2020-01-03 00:04:16,995834,Completed,1098,Application,42.346798,-71.093657,2020-01-03 00:08:18,2020-01-03 00:21:34,4.03,49.20,41743.0,NaN,2020-01-03 00:13:35,2020-01-03 00:20:08,1.20,6.56,5.0
4,2020-01-03,2020-01-03 01:56:18,995848,Completed,43,Application,42.326003,-71.082605,2020-01-03 02:07:14,2020-01-03 02:22:36,10.94,75.44,41747.0,NaN,2020-01-03 02:19:10,2020-01-03 02:27:31,1.57,8.35,5.0
5,2020-01-03,2020-01-03 19:00:44,995933,Completed,567,Application,42.333124,-71.102291,2020-01-03 19:03:29,2020-01-03 19:16:15,2.75,49.20,41750.0,NaN,2020-01-03 19:13:00,2020-01-03 19:20:34,1.11,7.56,5.0
6,2020-01-03,2020-01-03 19:49:58,996574,Completed,2778,Application,42.344116,-71.100349,2020-01-03 20:01:48,2020-01-03 20:19:03,11.82,49.20,41772.0,NaN,2020-01-03 20:11:27,2020-01-03 20:19:15,1.13,7.79,5.0
7,2020-01-03,2020-01-03 20:18:55,996808,Completed,2723,Application,42.346798,-71.093657,2020-01-03 20:46:13,2020-01-03 21:07:44,27.30,49.20,41785.0,NaN,2020-01-03 20:55:18,2020-01-03 21:05:53,1.12,10.58,5.0
8,2020-01-03,2020-01-03 20:44:27,997012,Completed,743,Application,42.328918,-71.101913,2020-01-03 21:10:29,2020-01-03 21:33:48,26.04,49.20,41793.0,NaN,2020-01-03 21:09:05,2020-01-03 21:16:52,1.29,7.79,5.0
9,2020-01-03,2020-01-03 21:16:57,997187,Completed,3928,Application,42.334283,-71.100857,2020-01-03 21:46:42,2020-01-03 22:04:47,29.74,49.20,41810.0,NaN,2020-01-03 21:38:43,2020-01-03 21:44:01,1.02,5.30,5.0


## Looking at some stats about some columns

In [3]:
for column in ['Request Status', 'Booking Method']:
    print(f'{column}:', df[column].unique())


for column in ['Time from request creation to planned pickup', 'Proposed Pickup Walk Distance']:
    print(f'{column}:', np.min(df[column]), np.max(df[column]))

Request Status: ['Completed' 'Out of Service Hours' 'Cancelled' 'Seat Unavailable'
 'Not Accepted' 'No-Showed' 'Other Error / Skip Billing']
Booking Method: ['Application' 'Agent']
Time from request creation to planned pickup: 0.25 58.55
Proposed Pickup Walk Distance: 0.0 1518.64


With the knowledge I got from the described categories, I will assume a few categories usefulness, uselessness, and figure out which ones I am unsure about

Useful:
- Request Status: This is the label column
- Original Planned Pickup/Dropoff: I know this is important to me when figuring out if I want to take a service, how long until it arrive and how long til ig et to my destination, especially in a time crunch
- Time From request creation to planned pickup: Longer waits may make cancelling more common 
- Proposed Pickup Walk Distance: If the walk to the pickup area is further than public transport or the housing, it probably isn't worth it.

Useless:
- Request Creation Date: I might come back to this as weekends/holidays might see a change, but otherwise I think it would be constant no matter the day
- Request ID: No relevance obviously
- Rider ID: Since I don't know how these are processed in the system, I don't want to use this. It might be helpful to learn an individual riders pattern, but I think a more general form of this model oblivious to rider ID would be good
- Booking Method: This doesn't give great information and the dichotomy seems to be from some internal code
- Ride ID: Ever changing, no real use or data given
- Cancellation Time: If you know this, it is already been cancelled, we want to predict before they cancel, so this is useless
- Actual Pickup Time: If they were actually picked up, then they didn't cancel, so won't be useful for predicting prior
- Actual Dropoff Time: Again no use as they can't cancel anymore
- Ride Duration: Again only hapens when not cancelled, so useless



Unsure:
- Creation Time: I can see how later in the night there might be less cancellations than earlier. Can be used in conjunction with proposed pickup time, but that is already handled by the Time from request creation to planned pickup so may be excess
- Destination Latitude/Longitude: These are in a pair and where a person is going may determine if they are within a faster walking distance or the such, I will keep considering this as explore the data. There may be a location that people like to go to but closes after a certain time and people don't realize.
- Ride Rating: Unsure if this is given by the rider or the quality of the ride before
- Ride Distance: Further away is probably less likely to cancel, might be useful, make sure it is in cancelled to


## Exploring the Unique Labels

In [4]:
df.loc[df['Request Status'] == 'Cancelled']

,Request Creation Date,Request Creation Time,Request ID,Request Status,Rider ID,Booking Method,Destination Lat,Destination Lng,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Ride ID,Cancellation Time,Actual Pickup Time,Actual Dropoff Time,Ride Distance,Ride Duration,Ride Rating (1-5)
2197,2020-01-01,2020-01-01 00:07:49,993928,Cancelled,3592,Application,42.343793,-71.098106,2020-01-01 00:22:43,2020-01-01 00:41:00,14.90,49.2,41575.0,2020-01-01 00:21:28,NaN,NaN,NaN,NaN,NaN
2205,2020-01-01,2020-01-01 01:17:55,993971,Cancelled,1896,Application,42.343243,-71.096042,2020-01-01 01:45:24,2020-01-01 02:06:51,27.47,49.2,41581.0,2020-01-01 01:18:06,NaN,NaN,NaN,NaN,NaN
2230,2020-01-01,2020-01-01 20:07:42,994036,Cancelled,2157,Application,42.333213,-71.102987,2020-01-01 20:11:59,2020-01-01 20:28:03,4.28,49.2,41600.0,2020-01-01 20:13:30,NaN,NaN,NaN,NaN,NaN
2244,2020-01-01,2020-01-01 21:58:36,994071,Cancelled,321,Application,42.333138,-71.097695,2020-01-01 22:00:26,2020-01-01 22:11:55,1.83,49.2,41611.0,2020-01-01 21:58:49,NaN,NaN,NaN,NaN,NaN
2254,2020-01-01,2020-01-01 23:07:34,994084,Cancelled,2120,Application,42.365391,-71.104751,2020-01-01 23:14:34,2020-01-01 23:33:26,7.00,98.4,41620.0,2020-01-01 23:07:43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57394,2020-12-28,2020-12-28 23:16:09,1871291,Cancelled,1483,Application,42.330087,-71.058081,2020-12-28 23:22:10,2020-12-28 23:35:07,6.00,16.4,93322.0,2020-12-28 23:16:51,NaN,NaN,NaN,NaN,NaN
57404,2020-12-29,2020-12-29 00:06:01,1871334,Cancelled,1369,Application,42.333508,-71.101820,2020-12-29 00:15:16,2020-12-29 00:22:05,9.23,16.4,93346.0,2020-12-29 00:11:41,NaN,NaN,NaN,NaN,NaN
57459,2020-12-30,2020-12-30 19:42:39,1871543,Cancelled,985,Application,42.345196,-71.090330,2020-12-30 19:45:04,2020-12-30 19:49:53,2.41,16.4,93412.0,2020-12-30 19:43:00,NaN,NaN,NaN,NaN,NaN
57488,2020-12-30,2020-12-30 23:56:25,1871610,Cancelled,36,Application,42.334390,-71.100892,2020-12-30 23:59:06,2020-12-31 00:04:22,2.67,16.4,93440.0,2020-12-30 23:56:40,NaN,NaN,NaN,NaN,NaN


Seeing this, I can remove ride rating and distance from my list. I like the idea of using longitude and latitude as well as time, so I believe it will be useful. Date and time will likely be useful and since I have a good range I think I can use it successfully in the model, but this might change later, so I will try it for now.

In [5]:
useful_columns = ['Request Status', 'Request Creation Time', 'Original Planned Pickup Time', 'Original Planned Dropoff Time', 
                    'Time from request creation to planned pickup', 'Proposed Pickup Walk Distance', 'Destination Lat', 'Destination Lng']
df = df[useful_columns]
df.head(10)

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
0,Completed,2020-01-01 04:43:01,2020-01-01 04:47:10,2020-01-01 05:04:05,4.14,72.16,42.326003,-71.082605
1,Completed,2020-01-01 21:53:47,2020-01-01 21:59:48,2020-01-01 22:14:41,6.00,49.20,42.322582,-71.109085
2,Completed,2020-01-02 00:55:31,2020-01-02 01:01:32,2020-01-02 01:10:56,6.00,49.20,42.329393,-71.104067
3,Completed,2020-01-03 00:04:16,2020-01-03 00:08:18,2020-01-03 00:21:34,4.03,49.20,42.346798,-71.093657
4,Completed,2020-01-03 01:56:18,2020-01-03 02:07:14,2020-01-03 02:22:36,10.94,75.44,42.326003,-71.082605
5,Completed,2020-01-03 19:00:44,2020-01-03 19:03:29,2020-01-03 19:16:15,2.75,49.20,42.333124,-71.102291
6,Completed,2020-01-03 19:49:58,2020-01-03 20:01:48,2020-01-03 20:19:03,11.82,49.20,42.344116,-71.100349
7,Completed,2020-01-03 20:18:55,2020-01-03 20:46:13,2020-01-03 21:07:44,27.30,49.20,42.346798,-71.093657
8,Completed,2020-01-03 20:44:27,2020-01-03 21:10:29,2020-01-03 21:33:48,26.04,49.20,42.328918,-71.101913
9,Completed,2020-01-03 21:16:57,2020-01-03 21:46:42,2020-01-03 22:04:47,29.74,49.20,42.334283,-71.100857


## Removing NaNs and figuring out label column
As we know, the label column has a few extra things beside completed and cancelled.

In [6]:
df['Request Status'].unique()

array(['Completed', 'Out of Service Hours', 'Cancelled',
       'Seat Unavailable', 'Not Accepted', 'No-Showed',
       'Other Error / Skip Billing'], dtype=object)

### I am going to explore if any of these sections have NaNs in them


In [7]:
df[df['Request Status'] == 'Out of Service Hours']

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
1693,Out of Service Hours,2020-10-30 19:00:16,2020-10-30 19:06:16,2020-10-30 19:14:08,6.0,16.4,42.331629,-71.099868
2218,Out of Service Hours,2020-01-01 16:00:28,NaN,NaN,NaN,NaN,42.316979,-71.082084
2219,Out of Service Hours,2020-01-01 17:09:06,NaN,NaN,NaN,NaN,42.346515,-71.093965
2220,Out of Service Hours,2020-01-01 18:30:46,NaN,NaN,NaN,NaN,42.342575,-71.097496
2269,Out of Service Hours,2020-01-02 16:04:14,NaN,NaN,NaN,NaN,42.328918,-71.101913
...,...,...,...,...,...,...,...,...
57409,Out of Service Hours,2020-12-29 16:16:04,NaN,NaN,NaN,NaN,42.344247,-71.099738
57410,Out of Service Hours,2020-12-29 16:54:59,NaN,NaN,NaN,NaN,42.352165,-71.123733
57411,Out of Service Hours,2020-12-29 18:05:46,NaN,NaN,NaN,NaN,42.345445,-71.100171
57412,Out of Service Hours,2020-12-29 18:09:43,NaN,NaN,NaN,NaN,42.342785,-71.120016


In [8]:
df[df['Request Status'] == 'Seat Unavailable']

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
2200,Seat Unavailable,2020-01-01 00:47:55,NaN,NaN,NaN,NaN,42.333912,-71.082244
2212,Seat Unavailable,2020-01-01 02:51:39,NaN,NaN,NaN,NaN,42.328918,-71.101913
2213,Seat Unavailable,2020-01-01 03:00:41,NaN,NaN,NaN,NaN,42.329564,-71.090312
2288,Seat Unavailable,2020-01-02 19:10:26,NaN,NaN,NaN,NaN,42.346596,-71.093430
2294,Seat Unavailable,2020-01-02 19:31:18,NaN,NaN,NaN,NaN,42.340470,-71.126600
...,...,...,...,...,...,...,...,...
57262,Seat Unavailable,2020-12-25 23:57:08,NaN,NaN,NaN,NaN,42.346085,-71.089560
57275,Seat Unavailable,2020-12-26 02:36:44,NaN,NaN,NaN,NaN,42.319461,-71.080917
57276,Seat Unavailable,2020-12-26 03:53:06,NaN,NaN,NaN,NaN,42.331546,-71.108291
57277,Seat Unavailable,2020-12-26 04:15:52,NaN,NaN,NaN,NaN,42.334390,-71.100892


In [9]:
df[df['Request Status'] == 'Not Accepted']

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
2201,Not Accepted,2020-01-01 01:01:35,2020-01-01 01:05:10,2020-01-01 01:17:16,3.58,49.20,42.328918,-71.101913
2202,Not Accepted,2020-01-01 01:02:00,2020-01-01 01:05:42,2020-01-01 01:17:45,3.69,49.20,42.328766,-71.101997
2206,Not Accepted,2020-01-01 01:22:27,2020-01-01 01:46:36,2020-01-01 02:06:03,24.16,229.60,42.342460,-71.101286
2215,Not Accepted,2020-01-01 05:20:42,2020-01-01 05:24:25,2020-01-01 05:37:51,3.72,45.92,42.348111,-71.087173
2216,Not Accepted,2020-01-01 05:30:56,2020-01-01 05:34:39,2020-01-01 05:47:15,3.71,49.20,42.329579,-71.104048
...,...,...,...,...,...,...,...,...
57505,Not Accepted,2020-12-31 19:04:46,2020-12-31 19:09:53,2020-12-31 19:19:35,5.09,16.40,42.328918,-71.101913
57507,Not Accepted,2020-12-31 19:06:50,2020-12-31 19:09:53,2020-12-31 19:22:00,3.03,16.40,42.326680,-71.104582
57522,Not Accepted,2020-12-31 21:35:34,2020-12-31 21:41:35,2020-12-31 21:50:37,6.00,16.40,42.328449,-71.092847
57523,Not Accepted,2020-12-31 21:52:29,2020-12-31 21:56:42,2020-12-31 22:10:42,4.17,16.40,42.346982,-71.122060


In [10]:
df[df['Request Status'] == 'No-Showed']

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
2233,No-Showed,2020-01-01 20:12:56,2020-01-01 20:35:28,2020-01-01 20:41:36,22.52,49.2,42.345196,-71.090330
2359,No-Showed,2020-01-02 21:50:19,2020-01-02 22:03:06,2020-01-02 22:11:48,12.78,49.2,42.346916,-71.093527
2360,No-Showed,2020-01-02 21:51:05,2020-01-02 22:03:06,2020-01-02 22:11:27,12.01,49.2,42.346953,-71.093557
2389,No-Showed,2020-01-02 23:57:44,2020-01-03 00:04:36,2020-01-03 00:24:21,6.86,49.2,42.329579,-71.104048
2475,No-Showed,2020-01-03 20:17:28,2020-01-03 20:46:13,2020-01-03 21:07:54,28.76,49.2,42.327999,-71.098267
...,...,...,...,...,...,...,...,...
57330,No-Showed,2020-12-27 20:21:31,2020-12-27 20:26:01,2020-12-27 20:35:35,4.47,16.4,42.332818,-71.096872
57403,No-Showed,2020-12-29 00:06:01,2020-12-29 00:15:16,2020-12-29 00:22:05,9.23,16.4,42.333508,-71.101820
57405,No-Showed,2020-12-29 00:06:01,2020-12-29 00:15:16,2020-12-29 00:21:19,9.23,16.4,42.334390,-71.100892
57408,No-Showed,2020-12-29 01:04:51,2020-12-29 01:11:05,2020-12-29 01:17:12,6.18,16.4,42.334283,-71.100857


In [11]:
df[df['Request Status'] == 'Other Error / Skip Billing']

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
2582,Other Error / Skip Billing,2020-01-04 00:38:09,2020-01-04 01:06:59,2020-01-04 01:28:48,28.83,49.2,42.331719,-71.109116
2583,Other Error / Skip Billing,2020-01-04 00:38:09,2020-01-04 01:06:59,2020-01-04 01:27:50,28.82,49.2,42.332409,-71.107195
2584,Other Error / Skip Billing,2020-01-04 00:38:10,2020-01-04 01:06:59,2020-01-04 01:28:48,28.81,49.2,42.331669,-71.109337
2585,Other Error / Skip Billing,2020-01-04 00:40:08,2020-01-04 01:08:01,2020-01-04 01:40:43,27.89,49.2,42.330090,-71.100697
2586,Other Error / Skip Billing,2020-01-04 00:41:59,2020-01-04 01:08:46,2020-01-04 01:29:06,26.77,49.2,42.331883,-71.109178
...,...,...,...,...,...,...,...,...
57198,Other Error / Skip Billing,2020-12-24 19:56:57,NaN,NaN,NaN,NaN,42.333508,-71.101820
57248,Other Error / Skip Billing,2020-12-25 21:17:23,NaN,NaN,NaN,NaN,42.337904,-71.088044
57355,Other Error / Skip Billing,2020-12-28 00:03:23,NaN,NaN,NaN,NaN,42.338720,-71.088400
57364,Other Error / Skip Billing,2020-12-28 03:35:56,NaN,NaN,NaN,NaN,42.311625,-71.093290


With this, I think I should only utilize Completed and Cancelled. No-Show could be considered cancelled, but it could also be someone is running late or forgot something, so I worry it could bias the data. Additionally, it is a relatively small portion of the dataset, so I don't believe it will be missed. the other four seem like they could be errors in the app and can't tell me if it was cancelled or not, so I will leave them out.

In [12]:
df = df[(df['Request Status'] == 'Completed') | (df['Request Status'] == 'Cancelled')]

In [13]:
df['Request Status'].unique()

array(['Completed', 'Cancelled'], dtype=object)

In [14]:
df

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng
0,Completed,2020-01-01 04:43:01,2020-01-01 04:47:10,2020-01-01 05:04:05,4.14,72.16,42.326003,-71.082605
1,Completed,2020-01-01 21:53:47,2020-01-01 21:59:48,2020-01-01 22:14:41,6.00,49.20,42.322582,-71.109085
2,Completed,2020-01-02 00:55:31,2020-01-02 01:01:32,2020-01-02 01:10:56,6.00,49.20,42.329393,-71.104067
3,Completed,2020-01-03 00:04:16,2020-01-03 00:08:18,2020-01-03 00:21:34,4.03,49.20,42.346798,-71.093657
4,Completed,2020-01-03 01:56:18,2020-01-03 02:07:14,2020-01-03 02:22:36,10.94,75.44,42.326003,-71.082605
...,...,...,...,...,...,...,...,...
57527,Completed,2020-12-31 23:55:54,2021-01-01 00:01:55,2021-01-01 00:11:44,6.00,16.40,42.335799,-71.077206
57528,Completed,2020-12-31 23:56:38,2021-01-01 00:01:55,2021-01-01 00:08:58,5.29,16.40,42.333508,-71.101820
57529,Completed,2020-12-31 23:57:17,2021-01-01 00:01:55,2021-01-01 00:15:48,4.63,16.40,42.326680,-71.104582
57530,Completed,2020-12-31 23:58:11,2021-01-01 00:01:55,2021-01-01 00:09:48,3.72,16.40,42.333355,-71.101913


In [15]:
df.isnull().values.any()

False

## Dealing with Date and Time
Since I have finished cleaning out my data and there are no Nulls left, this means I can process the date and time into something readable by Python. I will keep the month day, and request time. I will also calculate the length that the ride will take. This information will let my models know when the request happens, when pickup will happen (by the time difference) and when dropoff will happen (by the time distance again)

In [16]:
df['Request Creation Time'] = pd.to_datetime(df['Request Creation Time'], infer_datetime_format=True)
df['Original Planned Pickup Time'] = pd.to_datetime(df['Original Planned Pickup Time'], infer_datetime_format=True)
df['Original Planned Dropoff Time'] = pd.to_datetime(df['Original Planned Dropoff Time'], infer_datetime_format=True)

In [17]:
print(df['Request Creation Time'].head(10))
print(df['Original Planned Pickup Time'].head(10))
print(df['Original Planned Dropoff Time'].head(10))

0   2020-01-01 04:43:01
1   2020-01-01 21:53:47
2   2020-01-02 00:55:31
3   2020-01-03 00:04:16
4   2020-01-03 01:56:18
5   2020-01-03 19:00:44
6   2020-01-03 19:49:58
7   2020-01-03 20:18:55
8   2020-01-03 20:44:27
9   2020-01-03 21:16:57
Name: Request Creation Time, dtype: datetime64[ns]
0   2020-01-01 04:47:10
1   2020-01-01 21:59:48
2   2020-01-02 01:01:32
3   2020-01-03 00:08:18
4   2020-01-03 02:07:14
5   2020-01-03 19:03:29
6   2020-01-03 20:01:48
7   2020-01-03 20:46:13
8   2020-01-03 21:10:29
9   2020-01-03 21:46:42
Name: Original Planned Pickup Time, dtype: datetime64[ns]
0   2020-01-01 05:04:05
1   2020-01-01 22:14:41
2   2020-01-02 01:10:56
3   2020-01-03 00:21:34
4   2020-01-03 02:22:36
5   2020-01-03 19:16:15
6   2020-01-03 20:19:03
7   2020-01-03 21:07:44
8   2020-01-03 21:33:48
9   2020-01-03 22:04:47
Name: Original Planned Dropoff Time, dtype: datetime64[ns]


In [18]:
df.isnull().values.any()

False

In [19]:
# I am going to keep month, day, hour, and minute as I believe those are relevant. 
# Year will bias my data over many years and second just doesn't seem important.
df['Request Creation Month'] = df['Request Creation Time'].dt.month
df['Request Creation Day'] = df['Request Creation Time'].dt.day
df['Request Creation Hour'] = df['Request Creation Time'].dt.hour
df['Request Creation Minute'] = df['Request Creation Time'].dt.minute
df['Request Creation Day Of Week'] = df['Request Creation Time'].dt.dayofweek


In [20]:
df

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng,Request Creation Month,Request Creation Day,Request Creation Hour,Request Creation Minute,Request Creation Day Of Week
0,Completed,2020-01-01 04:43:01,2020-01-01 04:47:10,2020-01-01 05:04:05,4.14,72.16,42.326003,-71.082605,1,1,4,43,2
1,Completed,2020-01-01 21:53:47,2020-01-01 21:59:48,2020-01-01 22:14:41,6.00,49.20,42.322582,-71.109085,1,1,21,53,2
2,Completed,2020-01-02 00:55:31,2020-01-02 01:01:32,2020-01-02 01:10:56,6.00,49.20,42.329393,-71.104067,1,2,0,55,3
3,Completed,2020-01-03 00:04:16,2020-01-03 00:08:18,2020-01-03 00:21:34,4.03,49.20,42.346798,-71.093657,1,3,0,4,4
4,Completed,2020-01-03 01:56:18,2020-01-03 02:07:14,2020-01-03 02:22:36,10.94,75.44,42.326003,-71.082605,1,3,1,56,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57527,Completed,2020-12-31 23:55:54,2021-01-01 00:01:55,2021-01-01 00:11:44,6.00,16.40,42.335799,-71.077206,12,31,23,55,3
57528,Completed,2020-12-31 23:56:38,2021-01-01 00:01:55,2021-01-01 00:08:58,5.29,16.40,42.333508,-71.101820,12,31,23,56,3
57529,Completed,2020-12-31 23:57:17,2021-01-01 00:01:55,2021-01-01 00:15:48,4.63,16.40,42.326680,-71.104582,12,31,23,57,3
57530,Completed,2020-12-31 23:58:11,2021-01-01 00:01:55,2021-01-01 00:09:48,3.72,16.40,42.333355,-71.101913,12,31,23,58,3


In [21]:
# Get the time that the ride will be
df['Time from pickup to dropoff'] = (df['Original Planned Dropoff Time'] - df['Original Planned Pickup Time']).dt.total_seconds()/60

In [22]:
df['Time from pickup to dropoff'] = df['Time from pickup to dropoff'].round(2)

In [23]:
df

,Request Status,Request Creation Time,Original Planned Pickup Time,Original Planned Dropoff Time,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng,Request Creation Month,Request Creation Day,Request Creation Hour,Request Creation Minute,Request Creation Day Of Week,Time from pickup to dropoff
0,Completed,2020-01-01 04:43:01,2020-01-01 04:47:10,2020-01-01 05:04:05,4.14,72.16,42.326003,-71.082605,1,1,4,43,2,16.92
1,Completed,2020-01-01 21:53:47,2020-01-01 21:59:48,2020-01-01 22:14:41,6.00,49.20,42.322582,-71.109085,1,1,21,53,2,14.88
2,Completed,2020-01-02 00:55:31,2020-01-02 01:01:32,2020-01-02 01:10:56,6.00,49.20,42.329393,-71.104067,1,2,0,55,3,9.40
3,Completed,2020-01-03 00:04:16,2020-01-03 00:08:18,2020-01-03 00:21:34,4.03,49.20,42.346798,-71.093657,1,3,0,4,4,13.27
4,Completed,2020-01-03 01:56:18,2020-01-03 02:07:14,2020-01-03 02:22:36,10.94,75.44,42.326003,-71.082605,1,3,1,56,4,15.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57527,Completed,2020-12-31 23:55:54,2021-01-01 00:01:55,2021-01-01 00:11:44,6.00,16.40,42.335799,-71.077206,12,31,23,55,3,9.82
57528,Completed,2020-12-31 23:56:38,2021-01-01 00:01:55,2021-01-01 00:08:58,5.29,16.40,42.333508,-71.101820,12,31,23,56,3,7.05
57529,Completed,2020-12-31 23:57:17,2021-01-01 00:01:55,2021-01-01 00:15:48,4.63,16.40,42.326680,-71.104582,12,31,23,57,3,13.88
57530,Completed,2020-12-31 23:58:11,2021-01-01 00:01:55,2021-01-01 00:09:48,3.72,16.40,42.333355,-71.101913,12,31,23,58,3,7.88


In [24]:
# Now I can remove the time stamp columns as they are no longer useful
df.pop('Request Creation Time')
df.pop('Original Planned Pickup Time')
df.pop('Original Planned Dropoff Time')
df

,Request Status,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng,Request Creation Month,Request Creation Day,Request Creation Hour,Request Creation Minute,Request Creation Day Of Week,Time from pickup to dropoff
0,Completed,4.14,72.16,42.326003,-71.082605,1,1,4,43,2,16.92
1,Completed,6.00,49.20,42.322582,-71.109085,1,1,21,53,2,14.88
2,Completed,6.00,49.20,42.329393,-71.104067,1,2,0,55,3,9.40
3,Completed,4.03,49.20,42.346798,-71.093657,1,3,0,4,4,13.27
4,Completed,10.94,75.44,42.326003,-71.082605,1,3,1,56,4,15.37
...,...,...,...,...,...,...,...,...,...,...,...
57527,Completed,6.00,16.40,42.335799,-71.077206,12,31,23,55,3,9.82
57528,Completed,5.29,16.40,42.333508,-71.101820,12,31,23,56,3,7.05
57529,Completed,4.63,16.40,42.326680,-71.104582,12,31,23,57,3,13.88
57530,Completed,3.72,16.40,42.333355,-71.101913,12,31,23,58,3,7.88


In [25]:
print(df.isnull().values.any())
print(df['Request Status'].unique())

False
['Completed' 'Cancelled']


In [26]:
# since I believe I have isolated all the data I will need, I will save this non-normalized data to a file
df.to_csv('non-normalized data.csv', index=False)

# Normalizing Data

In [27]:
# Turn request status to 0 or 1. Cancelled being the 1 class we are trying to identify H1 and Completed being the 0 class that we assume everything to be H0
df['Request Status'] = [int(x) for x in df['Request Status'] == 'Cancelled']

In [28]:
df['Request Status'].unique()

array([0, 1], dtype=int64)

In [29]:
df

,Request Status,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng,Request Creation Month,Request Creation Day,Request Creation Hour,Request Creation Minute,Request Creation Day Of Week,Time from pickup to dropoff
0,0,4.14,72.16,42.326003,-71.082605,1,1,4,43,2,16.92
1,0,6.00,49.20,42.322582,-71.109085,1,1,21,53,2,14.88
2,0,6.00,49.20,42.329393,-71.104067,1,2,0,55,3,9.40
3,0,4.03,49.20,42.346798,-71.093657,1,3,0,4,4,13.27
4,0,10.94,75.44,42.326003,-71.082605,1,3,1,56,4,15.37
...,...,...,...,...,...,...,...,...,...,...,...
57527,0,6.00,16.40,42.335799,-71.077206,12,31,23,55,3,9.82
57528,0,5.29,16.40,42.333508,-71.101820,12,31,23,56,3,7.05
57529,0,4.63,16.40,42.326680,-71.104582,12,31,23,57,3,13.88
57530,0,3.72,16.40,42.333355,-71.101913,12,31,23,58,3,7.88


In [30]:
print(np.min(df['Destination Lat']), np.max(df['Destination Lat']))
print(np.min(df['Destination Lng']), np.max(df['Destination Lng']))

42.312190903 42.368296086
-71.127540842 -71.048911884


In [31]:
# Scale Data to either [0, 1] range for time and distance reducing down to just the decimals
cols_minmax = ['Time from request creation to planned pickup', 'Proposed Pickup Walk Distance', 'Request Creation Month', 
            'Request Creation Day', 'Request Creation Hour', 'Request Creation Minute', 'Request Creation Day Of Week', 'Time from pickup to dropoff']
minmax_scalar = MinMaxScaler()
df[cols_minmax] = minmax_scalar.fit_transform(df[cols_minmax])
# Shift but don't scale coordinates
df['Destination Lat'] = df['Destination Lat'] - 42
df['Destination Lng'] = df['Destination Lng'] - -71


In [33]:
print(df.isnull().values.any())
print(df['Request Status'].unique())

False
[0 1]


In [35]:
df.head(10)

,Request Status,Time from request creation to planned pickup,Proposed Pickup Walk Distance,Destination Lat,Destination Lng,Request Creation Month,Request Creation Day,Request Creation Hour,Request Creation Minute,Request Creation Day Of Week,Time from pickup to dropoff
0,0,0.066724,0.049327,0.326003,-0.082605,0.0,0.000000,0.173913,0.728814,0.333333,0.555201
1,0,0.098628,0.033632,0.322582,-0.109085,0.0,0.000000,0.913043,0.898305,0.333333,0.522655
2,0,0.098628,0.033632,0.329393,-0.104067,0.0,0.033333,0.000000,0.932203,0.500000,0.435227
3,0,0.064837,0.033632,0.346798,-0.093657,0.0,0.066667,0.000000,0.067797,0.666667,0.496969
4,0,0.183362,0.051570,0.326003,-0.082605,0.0,0.066667,0.043478,0.949153,0.666667,0.530472
5,0,0.042882,0.033632,0.333124,-0.102291,0.0,0.066667,0.826087,0.000000,0.666667,0.488992
6,0,0.198456,0.033632,0.344116,-0.100349,0.0,0.066667,0.826087,0.830508,0.666667,0.560466
7,0,0.463979,0.033632,0.346798,-0.093657,0.0,0.066667,0.869565,0.305085,0.666667,0.628590
8,0,0.442367,0.033632,0.328918,-0.101913,0.0,0.066667,0.869565,0.745763,0.666667,0.657307
9,0,0.505832,0.033632,0.334283,-0.100857,0.0,0.066667,0.913043,0.271186,0.666667,0.573708


In [34]:
df.to_csv('normalized data.csv', index=False)

Data is all normalized without error and nulls, should be ready to be used in a variety of models.